In [1]:
import os
import json
from QuaCer_CAnon.utils import *


/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvi

In [49]:
graph = {}
id2name = {}
id2source = {}
header2idx = {}
rel_id_counter = 0
rels = {}
edge2src = {}

def insert_edge(x, y, rel_name, line, header2idx, graph, id2name, 
                rels, rel_id_counter, edge2source, line_s, id2source, x_name, y_name , x_source, y_source):
    if rel_name not in rels:
        rels[rel_name] = rel_id_counter
        rel_id = 'RIDR' + str(rel_id_counter)
        rel_id_counter += 1
    else:
        rel_id = 'RIDR' + str(rels[rel_name])
    
    if rel_id in id2name:
        if id2name[rel_id] != rel_name:
            print(rel_id, id2name[rel_id], rel_name)
        assert id2name[rel_id] == rel_name
    id2name[rel_id] = rel_name
    if x not in graph:
        graph[x] = {}
    if x not in edge2src:
        edge2src[x] = {}
    graph[x][y] = rel_id
    edge2src[x][y] = line_s
    
    return graph, rel_id_counter, rels, edge2source, id2name, id2source

with open('kg.csv', 'r') as f:
    lines = f.readlines()
    headers = lines[0].strip().split(',')
    for i, header in enumerate(headers):
        header2idx[header] = i
    for line_s in lines[1:]:
        line = line_s.strip().split(',')
        line = [l.strip() for l in line]
        x = 'ENID'+str(line[header2idx['x_index']])
        y = 'ENID'+str(line[header2idx['y_index']])
        x_type = line[header2idx['x_type']]
        y_type = line[header2idx['y_type']]
        rel_name = line[header2idx['display_relation']]+'_'+y_type
        x_name = line[header2idx['x_name']]
        y_name = line[header2idx['y_name']]
        x_source = line[header2idx['x_source']]
        y_source = line[header2idx['y_source']]
        id2name[x] = x_name
        id2name[y] = y_name
        id2source[x] = x_source
        id2source[y] = y_source
        graph, rel_id_counter, rels, edge2source, id2name, id2source = insert_edge(x, y, rel_name, line, header2idx, graph, id2name, 
                    rels, rel_id_counter, edge2src, line_s, id2source, x_name, y_name, x_source, y_source)
        # if x_type != y_type:
        #     rel_name = line[header2idx['display_relation']]+'_'+x_type
        #     graph, rel_id_counter, rels, edge2source, id2name, id2source = insert_edge(y, x, rel_name, line, header2idx, graph, id2name, rels, 
        #                 rel_id_counter, edge2src, line_s, id2source, y_name, x_name, y_source, x_source)

In [50]:
entity_aliases = {}
relation_aliases = {}
for k, v in id2name.items():
    if k.startswith('RIDR'):
        relation_aliases[k] = [v]
    else:
        entity_aliases[k] = [v]

In [51]:
qa_algos = GraphAlgos(graph, entity_aliases, relation_aliases, allow_multiple_ans=True)
for i in range(5):
    query_results = qa_algos.generate_random_query(4, return_path=True)
    question_data, source, correct_answers, path_ids = query_results
    question = question_data[0]
    print(question)
    print([id2name[id] for id in path_ids], path_ids)

Path: ['ENID50401', 'ENID51250', 'ENID111868', 'ENID51249']
Possible Paths: 69
fungal-type cell wall (1->3)-beta-D-glucan metabolic process->(parent-child_biological_process) ->(parent-child_biological_process) ->(parent-child_biological_process) ->?
['fungal-type cell wall (1->3)-beta-D-glucan metabolic process', 'fungal-type cell wall (1->3)-beta-D-glucan biosynthetic process', 'ascospore wall (1->3)-beta-D-glucan biosynthetic process', 'ascospore wall (1->3)-beta-D-glucan metabolic process'] ['ENID50401', 'ENID51250', 'ENID111868', 'ENID51249']
Path: ['ENID88166', 'ENID25127']
Possible Paths: 2
Triangular epiphysis of the distal phalanx of the hallux->(parent-child_effect/phenotype) ->?
['Triangular epiphysis of the distal phalanx of the hallux', 'Triangular epiphyses of the hallux'] ['ENID88166', 'ENID25127']
Path: ['ENID111924', 'ENID51288']
Possible Paths: 1
penetration hypha formation->(parent-child_biological_process) ->?
['penetration hypha formation', 'formation of infection 

In [47]:
verts = qa_algos.get_best_vertices(1000)
for vert in verts[:100]:
    print(len(graph[vert]), vert, id2name[vert], id2source[vert])

17355 ENID129373 multi-cellular organism UBERON
16855 ENID64546 small intestine UBERON
16743 ENID63376 testis UBERON
16742 ENID66050 fallopian tube UBERON
16619 ENID64796 prostate gland UBERON
16600 ENID64544 spleen UBERON
16542 ENID63223 intestine UBERON
16513 ENID69442 dorsolateral prefrontal cortex UBERON
16476 ENID63500 esophagus UBERON
16473 ENID63423 stomach UBERON
16456 ENID64485 thyroid gland UBERON
16420 ENID63181 adult mammalian kidney UBERON
16209 ENID64798 adrenal gland UBERON
16142 ENID63604 colon UBERON
16048 ENID63501 saliva-secreting gland UBERON
15971 ENID63674 cortex of kidney UBERON
15959 ENID63359 prefrontal cortex UBERON
15918 ENID63745 myometrium UBERON
15888 ENID64311 frontal cortex UBERON
15842 ENID129374 female gonad UBERON
15806 ENID64523 heart left ventricle UBERON
15775 ENID63704 urinary bladder UBERON
15644 ENID64487 lung UBERON
15635 ENID63426 heart UBERON
15514 ENID63744 endometrium UBERON
15453 ENID63136 lymph node UBERON
15386 ENID63240 blood UBERON
153

In [54]:
start_id = 27160
verts = {}
add = 0
while add < 100:
    start_id += add
    add +=1
    vert = 'ENID' + str(start_id)
    verts[vert] = len(graph[vert])
verts_l = sorted(list(verts.keys()), key=lambda x: x[1], reverse=True)
for vert in verts_l:
    print(vert, len(graph[vert]))
    break

ENID27160 12


In [55]:
en_id = 'ENID27160'
all_path_ids = []
print(len(graph[en_id]), en_id, id2name[en_id], id2source[en_id])
for i in range(5):
    query_results = qa_algos.generate_random_query(4, return_path=True,source=en_id)
    question_data, source, correct_answers, path_ids = query_results
    question = question_data[0]
    print(question) 
    print([id2name[id] for id in path_ids])
    all_path_ids.append(path_ids)

12 ENID27160 congenital stationary night blindness autosomal dominant MONDO_grouped
Path: ['ENID27160', 'ENID22084']
Possible Paths: 3
congenital stationary night blindness autosomal dominant->(associated with_gene/protein) ->?
['congenital stationary night blindness autosomal dominant', 'PDE6B']
Path: ['ENID27160', 'ENID6417']
Possible Paths: 3
congenital stationary night blindness autosomal dominant->(associated with_gene/protein) ->?
['congenital stationary night blindness autosomal dominant', 'RHO']
Path: ['ENID27160', 'ENID35655', 'ENID27161', 'ENID38855']
Possible Paths: 106
congenital stationary night blindness autosomal dominant->(parent-child_disease) ->(parent-child_disease) ->(parent-child_disease) ->?
['congenital stationary night blindness autosomal dominant', 'autosomal dominant disease', 'autosomal dominant nonsyndromic deafness', 'postlingual non-syndromic genetic deafness']
Path: ['ENID27160', 'ENID35655', 'ENID28197']
Possible Paths: 108
congenital stationary night bl

In [53]:
path_ids = all_path_ids[2]
for i in range(len(path_ids)-1):
    node1 = path_ids[i]
    node2 = path_ids[i+1]
    print(edge2src[node1][node2])

anatomy_protein_present,expression present,64546,2108,anatomy,small intestine,UBERON,7059,3559,gene/protein,IL2RA,NCBI

bioprocess_protein,interacts with,7059,3559,gene/protein,IL2RA,NCBI,112177,42104,biological_process,positive regulation of activated T cell proliferation,GO

